In [ ]:
%matplotlib inline

# Robust OPT - Control variate - Quadratic function.


In [ ]:
from __future__ import annotations

from gemseo import configure_logger
from gemseo.algos.design_space import DesignSpace
from gemseo.algos.parameter_space import ParameterSpace
from gemseo.disciplines.analytic import AnalyticDiscipline

from gemseo_umdo.scenarios.umdo_scenario import UMDOScenario

configure_logger()

Firstly,
we define an [AnalyticDiscipline][gemseo.disciplines.analytic.AnalyticDiscipline]
implementing the random function $f(x,U)=(x+U)^2$:



In [ ]:
discipline = AnalyticDiscipline({"y": "(x+u)**2"}, name="quadratic_function")

where $x$ belongs to the interval $[-1,1]$:



In [ ]:
design_space = DesignSpace()
design_space.add_variable("x", l_b=-1, u_b=1.0, value=0.5)

and $U$ is a standard Gaussian variable:



In [ ]:
uncertain_space = ParameterSpace()
uncertain_space.add_random_variable("u", "OTNormalDistribution")

Then,
we build a [UMDOScenario][gemseo_umdo.scenarios.umdo_scenario.UMDOScenario]
to minimize a control variate-based estimation
of the expectation $\mathbb{E}[Y]$ where $Y=f(x,U)$:



In [ ]:
scenario = UMDOScenario(
    [discipline],
    "DisciplinaryOpt",
    "y",
    design_space,
    uncertain_space,
    "Mean",
    statistic_estimation="ControlVariate",
    statistic_estimation_parameters={"n_samples": 50},
)

We execute it with a gradient-free optimizer:



In [ ]:
scenario.execute({"algo": "NLOPT_COBYLA", "max_iter": 100})

and plot the history:



In [ ]:
scenario.post_process("OptHistoryView", save=True, show=True)

Notice that the numerical solution is equal to $(x^*,f^*)=(0,1)$ as expected
from the expression of the statistic: $\mathbb{E}[Y]=\mathbb{E}[(x+U)^2]=x^2+1$.

